In [4]:
import torch
import torchvision.transforms as transforms
import numpy as np
from scipy import ndimage

import config
import model_batchnormDP0
import model_batchnormDP05
import model_dropout0
import model_dropout05

In [1]:
# NET ONE
log = torch.load('logs/batchnormDP0.pth')
tokens = len(log['vocab']['question']) + 1
net1 = torch.nn.DataParallel(model_batchnormDP0.Net(tokens)).cuda()
net1.load_state_dict(log['weights'])
# NET TWO
log = torch.load('logs/batchnormDP05.pth')
tokens = len(log['vocab']['question']) + 1
net2 = torch.nn.DataParallel(model_batchnormDP05.Net(tokens)).cuda()
net2.load_state_dict(log['weights'])
# NET THREE
log = torch.load('logs/dropout0.pth')
tokens = len(log['vocab']['question']) + 1
net3 = torch.nn.DataParallel(model_dropout0.Net(tokens)).cuda()
net3.load_state_dict(log['weights'])
# NET FOUR
log = torch.load('logs/dropout05.pth')
tokens = len(log['vocab']['question']) + 1
net4 = torch.nn.DataParallel(model_dropout05.Net(tokens)).cuda()
net4.load_state_dict(log['weights'])

In [5]:
for net_i, net in enumerate([net1, net2, net3, net4]):
    # pass image through network
    transform = transforms.Compose([
                transforms.Scale(config.image_size),
                transforms.CenterCrop(config.image_size),
                transforms.ToTensor(),
            ])
    
    transform2 = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5]),
            ])
    I_tens = transform(I) 
    background = tens_to_img(I_tens)
    q = q_s[index]
    v = v_s[index]
    a = a_s[index]
    q_len = q_lens[index]

    net.eval()
    hq = net.module.text(q, list(q_len.data))
    hv = v/(v.norm(p=2, dim=1, keepdim=True).expand_as(v) + 1e-8)
    ha = net.module.attention(hv, hq)
    hv, attent = apply_attention(hv, ha)

    # get glimpse 1
    tens = attent[0, 0, 0, :]
    tens.shape[0]
    attent_img = tens.view(14,14).to('cpu').detach().numpy()
    up_img = ndimage.zoom(attent_img, 32, order=0)
    blur = ndimage.gaussian_filter(up_img, sigma=10)
    blur3 = 191*blur
    overlay = np.stack((blur3,blur3,blur3), axis=2)
    glimpse1 = cv2.addWeighted(background, 0.4,overlay,0.1,0)

    # get glimpse 2
    tens = attent[0, 1, 0, :]
    tens.shape[0]
    attent_img = tens.view(14,14).to('cpu').detach().numpy()
    up_img = ndimage.zoom(attent_img, 32, order=0)
    blur = ndimage.gaussian_filter(up_img, sigma=10)
    blur3 = 191*blur
    overlay = np.stack((blur3,blur3,blur3), axis=2)
    glimpse2 = cv2.addWeighted(background, 0.4,overlay,0.1,0)

    white = [255,255,255] 

    glimpse1=cv2.copyMakeBorder(glimpse1,5,5,5,5,cv2.BORDER_CONSTANT,value=white)
    glimpse2=cv2.copyMakeBorder(glimpse2,5,5,5,5,cv2.BORDER_CONSTANT,value=white)

    allglimpses.append([glimpse1, glimpse2])
        
    
    # answers 
    out = net(v, q, q_len)
    _, answer = out.data.cpu().max(dim=1)
    answ = (answer.view(-1))
    ans_voc = dict((v,k) for k,v in datavqa.answer_to_index.items())
    
    if net_i == 0:
        question = ""
        for i in range(question_len):
            print(rev_voc[question_ind[i]], end=' ')
            question = question + rev_voc[question_ind[i]] + ' '
        question = question + '?'    
        print("?")
    print("Network ({0}) answer: {1}".format(netname[net_i], ans_voc[answ.item()]))


/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


NameError: name 'I' is not defined